In [ ]:
from sqlalchemy import select, update, and_, cast, func, Integer, insert

In [ ]:
with session_factory() as session:
    query = select(SalesReportOrm)
    # r = session.get(SalesReportOrm, ident=1)  # Способ получения сущности, работает только для одной записи
    r = session.execute(query).scalars().all()[:10]
    # mappings делает сопоставление поля - значение, без него возвращается просто кортеж значений
    # Метод scalars() используется, когда мы ожидаем получить одну колонку результата, а не несколько полей. 
    # Например, когда мы запрашиваем всю модель (в нашем случае SalesReportOrm) или одно конкретное поле (например, SalesReportOrm).

    # r = session.execute(query).scalar()  
    # Метод scalar() используется, когда мы ожидаем одну запись и одно поле в результате запроса. Если запрос вернет больше одной строки, метод выбросит исключение.
    print('='*100)
    print(r[0].user_id)
    print('='*100)

[<Object: SalesReportOrm | Attributes: user_id=1, period_from=datetime.date(2024, 11, 21), period_to=datetime.date(2024, 12, 10), total_orders=31, total_revenue=164359.26, avg_order_value=5301.91, top_product_name='Робот-пылесос', top_product_sales=31565.96, cancelled_orders=3, successful_orders=19, returned_orders=3, payment_method_stats={'card': 100574.38, 'cash': 8552.81, 'online': 55232.08}, category_breakdown={'Умный дом': 54062.75, 'Транспорт': 7863.63, 'Аксессуары': 59397.57, 'Электроника': 43035.31}, sales_by_day={'2024-11-21': 6111.63, '2024-11-22': 5632.97, '2024-11-23': 1856.98, '2024-11-24': 1974.96, '2024-11-25': 519.76, '2024-11-26': 6089.54, '2024-11-27': 4822.54, '2024-11-28': 6248.29, '2024-11-29': 553.94, '2024-11-30': 3486.97, '2024-12-01': 404.54, '2024-12-02': 6268.0, '2024-12-03': 1320.98, '2024-12-04': 7699.34, '2024-12-05': 3756.55, '2024-12-06': 7940.49, '2024-12-07': 2260.07, '2024-12-08': 3344.26, '2024-12-09': 1357.87, '2024-12-10': 3854.09}, comment='Провер

In [ ]:
def education_examples():
    with session_factory() as session:
        user = session.get(UserOrm, 3)  # При обновлении через get будет 2 запроса, на получение и обновление
        user.name=random.choice(['Бобер', 'Цыпа', 'Голубио'])
        user.fullname=random.choice(['Бобер', 'Цыпа', 'Голубио'])
        # session.flush()  сохраняет изменения в бд, но не завершает сессию как commit
        session.expire(user)  # сброс незакоммиченных изменений объекта user
        session.expire_all()  # сброс всех незакоммиченных изменений в сессии
        print(user.fullname)  # после expire\expire_all при попытке доступа к полям объекта будет выполнен новый запрос к БД
        session.refresh(user)  # делает запрос в бд на получение актуальных значений

In [ ]:
from app.users.dao import UserDAO
from app.users.schemas import UserInDB
import random

# id: dict = UserDAO.create(username=f"spongebob{round(random.random(), 3)}", fullname="Spongebob Squarepants", password='acuna_matata')
# UserDAO.create_bulk([{'username': "spongebo221b", 'fullname':"Spongebob Squarepants"}, {'username': "spong11ebo1b", 'fullname':"Spongebob Squarepants"}])
# UserDAO.delete(id=69)
print('='*100)
user = UserDAO.find_one_or_none(id=3)
print(UserInDB(**user))
UserDAO.is_valid_password('acuna_matata', user['password'])

2025-08-20 14:46:21,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-20 14:46:21,087 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".username, "user".password, "user".fullname, "user".email, "user".disabled, "user".created_at, "user".updated_at 
FROM "user" 
WHERE "user".id = %(id_1)s::INTEGER
2025-08-20 14:46:21,090 INFO sqlalchemy.engine.Engine [cached since 2272s ago] [SQL parameters hidden due to hide_parameters=True]
2025-08-20 14:46:21,092 INFO sqlalchemy.engine.Engine ROLLBACK
username='spongebob0.077' fullname='Spongebob Squarepants' email=None disabled=False hashed_password='$2b$12$gwGmvdhGNO2FW1pqqSQpOOgQSmX5jHpGM2Zv8TeK4shy9kFm0V8ui'


True

In [ ]:
import bcrypt
password = '$2b$12$gwGmvdhGNO2FW1pqqSQpOOgQSmX5jHpGM2Zv8TeK4shy9kFm0V8ui'.encode()
# Hash a password for the first time, with a randomly-generated salt
hashed = bcrypt.hashpw(password, bcrypt.gensalt())
print(hashed)
# Check that an unhashed password matches one that has previously been
# hashed
if bcrypt.checkpw(password, hashed):
    print("It Matches!")
else:
    print("It Does not Match :(")

b'$2b$12$zwb/ZwhPiwc1Fayz3KtcMuPpxjSmN3TmtlU5MjspCroFSH2sdrRxC'
It Matches!


In [ ]:
import random
from app.users.dao import UserDAO


ids: dict = UserDAO.create_bulk([
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
    {'username': f"spongebob{round(random.random(), 3)}", 'fullname': "Spongebob Squarepants", 'password': 'acuna_matata'},
])
print('='*100)
print(ids)
for id in ids:
    user = UserDAO.find_one_or_none(id=id)
    user_pass = user.hashed_password
    print('='*100)
    print(user_pass)
    print(UserDAO.is_valid_password('acuna_matata', user_pass))

2025-09-14 21:42:46,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-14 21:42:46,999 INFO sqlalchemy.engine.Engine INSERT INTO "user" (username, password, fullname, disabled) VALUES (%(username_m0)s::VARCHAR, %(password_m0)s::VARCHAR, %(fullname_m0)s::VARCHAR, %(disabled)s), (%(username_m1)s::VARCHAR, %(password_m1)s::VARCHAR, %(fullname_m1)s::VARCHAR, %(disabled_m1)s), (%(username_m2)s::VARCHAR, %(password_m2)s::VARCHAR, %(fullname_m2)s::VARCHAR, %(disabled_m2)s), (%(username_m3)s::VARCHAR, %(password_m3)s::VARCHAR, %(fullname_m3)s::VARCHAR, %(disabled_m3)s) RETURNING "user".id
2025-09-14 21:42:47,000 INFO sqlalchemy.engine.Engine [no key 0.00102s] [SQL parameters hidden due to hide_parameters=True]
2025-09-14 21:42:47,003 INFO sqlalchemy.engine.Engine COMMIT
[14, 15, 16, 17]
2025-09-14 21:42:47,013 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-14 21:42:47,015 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".username, "user".password, "user".fullname, "user".e

In [ ]:
from app.users.schemas import UserInDB
from fastapi import Depends
from typing import Annotated


ALGORITHM = 'HS256'
ACCESS_TOKEN_EXPIRE_MINUTES = 30


def verify_password(password: str, hashed_password: str) -> bool:
    return True


def get_user(username: str) -> UserInDB | None:
    return UserInDB(
        username=username,
        fullname='sfdds',
        email='1123123',
        disabled=False,
        hashed_password='sdfdsfsfsdfsf'
    )


def authenticate_user(user: Annotated[UserInDB, Depends(get_user)]) -> UserInDB | None:
    print(user)
    if user and verify_password('password', user.hashed_password):
        return user


authenticate_user('132131230')

132131230


AttributeError: 'str' object has no attribute 'hashed_password'

In [ ]:
from app.pdf_reports.dao import ReportDAO, SalesReportDAO
from datetime import date

sales = SalesReportDAO.find_all(period_from=date(2024,11,7), period_to=date(2025, 7,5))

TypeError: '>=' not supported between instances of 'str' and 'datetime.date'

In [2]:
# Когда сложно выбрать, кто твой любимый пользователь... Эд или Венди 🤔
from sqlalchemy import or_, and_
from app.pdf_reports.models import ReportOrm, SalesReportOrm
from app.pdf_reports.dao import ReportDAO, SalesReportDAO

ReportDAO.update(1, file_path='file_path', status=ReportStatusEnum.done)

# sales = SalesReportDAO.find_all_where(
#     and_(SalesReportOrm.period_from >= date(2024,11,7), SalesReportOrm.period_to <= date(2025, 7,5))
# )
# len(sales)